# Imports

In [10]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', SyntaxWarning)

from satpy.scene import Scene
from satpy.utils import debug_on

from datetime import datetime

from glob import glob

import hvplot.xarray

import panel as pn

## Starting to create satpy scenes

In [2]:
sat_files = glob("input/G16_ABI-L1b-RadF/*")
sat_files

['input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C16_G16_s20232561536173_e20232561538557_c20232561539036.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C13_G16_s20232561536173_e20232561538557_c20232561539004.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C14_G16_s20232561536173_e20232561538546_c20232561538592.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C06_G16_s20232561536173_e20232561538551_c20232561539011.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C01_G16_s20232561536173_e20232561538548_c20232561538585.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C12_G16_s20232561536173_e20232561538552_c20232561539031.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C05_G16_s20232561536173_e20232561538546_c20232561539016.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C03_G16_s20232561536173_e20232561538546_c20232561538589.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C02_G16_s20232561536173_e20232561538546_c20232561538578.nc',
 'input/G16_ABI-L1b-RadF/OR_ABI-L1b-RadC-M6C04_G16_s202325615361

In [3]:
scn = Scene(filenames = sat_files, reader='abi_l1b')

dataset_names = scn.all_dataset_names()

#print(dataset_names)

In [10]:
dataset_names

['C01',
 'C02',
 'C03',
 'C04',
 'C05',
 'C06',
 'C07',
 'C08',
 'C09',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16']

In [12]:
components = scn.available_composite_names()

In [4]:
scn.load([f'C{x:02d}' for x in range(1, 17)])

In [29]:
band_select = pn.widgets.Select(name='Band name', options=dataset_names)

In [30]:
band_select_components = pn.widgets.Select(name='Component name', options=components)

In [32]:
band_select_components

Select(name='Component name', options=['24h_microphysics', ...], value='24h_microphysics')

In [14]:
def get_singleband_plot(bandname=None):
    
    scn.load([bandname])
    result = scn[bandname]
    interactive_plot = result.hvplot.image().opts(title=bandname)

    return interactive_plot

In [21]:
band_iplot = pn.bind(get_singleband_plot,bandname=band_select)
component_iplot = pn.bind(get_singleband_plot, bandname=band_select_components)

In [26]:
template = pn.template.MaterialTemplate(
    #site="",
    title="Visualize GOES-R data",
)
template.main.append(
         pn.Row(
            band_select,
             band_iplot,
             #band_select_components,
             #component_iplot
            #sizing_mode='stretch_both'
            )
        )

In [ ]:
print(scn.available_composite_names())

In [ ]:
bandname = 'airmass'
scn.load([bandname])
result = scn[bandname]


In [ ]:
scn.show(bandname)

In [ ]:
scn[bandname].load()

In [ ]:
result

In [ ]:
result.data.plot.imshow()

In [ ]:
da = result.data.load()

In [ ]:
da["bands"] = [0,1,2]

In [ ]:
import numpy as np

In [ ]:
np.floor(da*255)

In [ ]:
np.floor(da*255).hvplot.rgb(x='x',y='y',bands='bands',colorbar=True,rasterize=True,dynamic=True)

In [ ]:
from satpy.composites import get_enhanced_image

result = get_enhanced_image(scn[bandname])


In [11]:
import pandas as pd
import hvplot.pandas
import numpy as np


In [12]:
dt_input = pn.widgets.DatetimeInput(name='Datetime Input',value=datetime(2019, 2, 8))
dt_input

DatetimeInput(name='Datetime Input', type=<class 'datetime.datetime'..., value=datetime.datetime(2019, ...)

In [13]:
def date_print(date=None):
    
    print(date)

    index = pd.date_range('1/1/2000', periods=1000)
    df = pd.DataFrame(np.random.randn(1000, 4), index=index, columns=list('ABCD')).cumsum()
    
    retplot = df.hvplot(title=date.strftime('%Y-%m-%d %H:%M:%S'))

    return retplot
    
dummypane = pn.bind(date_print,date=dt_input)

In [14]:
template = pn.template.MaterialTemplate(
    #site="",
    title="Visualize GOES-R data",
)
# default='%Y-%m-%d %H:%M:%S'
template.main.append(
         pn.Row(
            dt_input,
             dummypane,
            #band_select,
            #band_iplot,
             #band_select_components,
             #component_iplot
            sizing_mode='stretch_both'
            )
        )

2019-02-08 00:00:00


In [ ]:
template#.servable()